# Huawei Research France

In [8]:
import pandas as pd
import numpy as np
import rampwf as rw
import datetime
import time
import importlib

In [9]:
import extract
from extract import PrepareExtractor
from DataCleaner import DataCleaner
importlib.reload(extract)

<module 'extract' from 'C:\\Users\\Daniel\\Desktop\\projets\\Huawei\\extract.py'>

In [3]:
problem = rw.utils.assert_read_problem()

Ajout des données

In [4]:
X_train, y_train = problem.get_train_data()
X_test, y_test = problem.get_test_data()

Train data
Optical Dataset composed of
46110 source samples
50862 source background samples
438 target labeled samples
8202 target unlabeled samples
29592 target background samples
 Optical Dataset labels composed of
46110 labels of source samples
438 labels of target samples

Test data
Optical Dataset composed of
0 source samples
0 source background samples
17758 target labeled samples
0 target unlabeled samples
47275 target background samples
 Optical Dataset labels composed of
0 labels of source samples
17758 labels of target samples



In [5]:
data_cleaner = DataCleaner(drop_olt_recv=True)
X_temp = data_cleaner.clean_data(X_train.source)

In [6]:
prep = PrepareExtractor()
temp, tempp = prep.get_data(X_temp, size_sample=200, resample={'unit': 'D', 'func': 'mean'}, name=None)

Temps :  1.0479333400726318


In [7]:
temp.head(2)

,2000-01-01 00:00:00 current,2000-01-01 00:00:00 err_down_bip,2000-01-01 00:00:00 err_up_bip,2000-01-01 00:00:00 rdown,2000-01-01 00:00:00 recv,2000-01-01 00:00:00 rup,2000-01-01 00:00:00 send,2000-01-01 00:00:00 temp,2000-01-01 00:00:00 volt,2000-01-02 00:00:00 current,...,2000-01-06 00:00:00 volt,2000-01-07 00:00:00 current,2000-01-07 00:00:00 err_down_bip,2000-01-07 00:00:00 err_up_bip,2000-01-07 00:00:00 rdown,2000-01-07 00:00:00 recv,2000-01-07 00:00:00 rup,2000-01-07 00:00:00 send,2000-01-07 00:00:00 temp,2000-01-07 00:00:00 volt
0,14.0,408.0,31.968750,0.820359,-30.465311,0.041641,2.264688,45.583332,3279.791748,14.000000,...,3280.0,14.052083,408.0,21.781250,1.282615,-30.459999,0.056198,2.263125,45.906250,3279.791748
0,11.0,0.0,1.052083,0.036052,-28.530626,0.003604,2.132396,41.947918,3280.000000,10.958333,...,3280.0,10.572917,0.0,0.822917,0.094948,-28.271250,0.004729,2.154896,40.802082,3280.000000


### Classification

Vous devez soumettre un extracteur de caractéristiques et un classificateur. La fonction transform de l'extracteur de caractéristiques est exécutée sur chaque donnée d'entrée (cible, source, bkg) et les tableaux résultants sont passés aux fonctions fit et predict du classificateur. L'extracteur de caractéristiques du kit de départ remplace nans par zéro, et aplatit la matrice en **(sample, 6720)**.

In [ ]:
trained_workflow = problem.workflow.train_submission('submissions/prepare_rf', X_train, y_train)
y_test_pred = problem.workflow.test_submission(trained_workflow, X_test)

### Scores

In [ ]:
ap    = problem.score_types[0]
rec5  = problem.score_types[1]
rec10 = problem.score_types[2]
rec20 = problem.score_types[3]
acc   = problem.score_types[4]
auc   = problem.score_types[5]

In [ ]:
print('ap test score    = {}'.format(ap(y_test.target, y_test_pred[:,1])))
print('rec5 test score  = {}'.format(rec5(y_test.target, y_test_pred[:,1])))
print('rec10 test score = {}'.format(rec10(y_test.target, y_test_pred[:,1])))
print('rec20 test score = {}'.format(rec20(y_test.target, y_test_pred[:,1])))
print('acc test score   = {}'.format(acc(y_test.target, y_test_pred.argmax(axis=1))))
print('auc test score   = {}'.format(auc(y_test.target, y_test_pred[:,1])))

### Cross validation

Nous utilisons une validation croisée dix fois (stratifiée lorsque les étiquettes sont disponibles) pour tous les ensembles de données. Dans chaque split, 20% des instances sont dans l'ensemble de validation, à l'exception des données cibles étiquetées qui servent principalement à la validation (pour obtenir une estimation non biaisée des scores de test, évalués entièrement sur des échantillons cibles étiquetés). Nous plaçons vingt points cibles étiquetés dans les splits d'entraînement. La raison en est que lorsque nous étendons nos services à large bande à la ville B, nous pouvons obtenir rapidement un petit ensemble de données étiquetées, mais nous aimerions déployer notre détecteur de défaillance sans attendre deux mois pour recueillir des données comparables à celles de la ville A.

Le schéma de validation croisée (voir `problem.get_cv`) est implémenté dans la classe `TLShuffleSplit` de `external_imports.utils.cv.py`, si vous voulez y regarder de plus près.

Vous êtes libre de jouer avec la coupure train/test et la validation croisée lors du développement de vos modèles mais sachez que nous utiliserons la même configuration sur le serveur officiel que celle du kit RAMP (sur un ensemble différent de quatre campagnes qui ne sera pas disponible pour vous).

La cellule suivante passe par les mêmes étapes que le script d'évaluation officiel (`ramp-test`).

In [ ]:
splits = problem.get_cv(X_train, y_train)

In [ ]:
splits = problem.get_cv(X_train, y_train)

y_test_preds = []
for fold_i, (train_is, valid_is) in enumerate(splits):
    trained_workflow = problem.workflow.train_submission(
        'submissions/starting_kit', X_train, y_train, train_is)
    X_fold_train = X_train.slice(train_is)
    X_fold_valid = X_train.slice(valid_is)
    
    y_train_pred = problem.workflow.test_submission(trained_workflow, X_fold_train)
    y_valid_pred = problem.workflow.test_submission(trained_workflow, X_fold_valid)
    y_test_pred = problem.workflow.test_submission(trained_workflow, X_test)
    print('-------------------------------------')
    print('training ap on fold {} = {}'.format(
        fold_i, ap(y_train.slice(train_is).target, y_train_pred[:,1])))
    print('validation ap on fold {} = {}'.format(
        fold_i, ap(y_train.slice(valid_is).target, y_valid_pred[:,1])))
    print('test ap on fold {} = {}'.format(fold_i, ap(y_test.target, y_test_pred[:,1])))
    
    y_test_preds.append(y_test_pred)

Nous calculons à la fois le score moyen du test et le score de la mise en sac de vos dix modèles. Le classement officiel sera déterminé par le score de test mis en sac (sur des ensembles de données différents de ceux dont vous disposez). Votre score public sera le score de validation mis en sac (le calcul de la moyenne est [légèrement plus compliqué](https://github.com/paris-saclay-cds/ramp-workflow/blob/master/rampwf/utils/combine.py#L56) car nous devons nous occuper correctement des masques de validation croisée). 

In [ ]:
bagged_y_pred = np.array(y_test_preds).mean(axis=0)
print('Mean ap score = {}'.format(
    np.mean([ap(y_test.target, y_test_pred[:,1]) for y_test_pred in y_test_preds])))
print('Bagged ap score = {}'.format(
    ap(y_test.target, np.array([y_test_pred for y_test_pred in y_test_preds]).mean(axis=0)[:,1])))

## Exemple submissions

Outre le kit de départ, nous vous proposons deux autres exemples de soumissions. L'extracteur de caractéristiques est le même dans les trois. `source_rf` est similaire au kit de départ, mais utilise des arbres plus nombreux et plus profonds, pour obtenir un meilleur score. `target_rf` est une autre soumission extrême qui utilise seulement l'instance d'entraînement de la cible (peu) étiquetée pour apprendre un classificateur. Il a une performance légèrement moins bonne que `source_rf` ce qui signifie que les données sources améliorent le classificateur même si les distributions sources et cibles sont différentes.

### Resultats:
|          | ap             | rec-5         | rec-10         | rec-20         | acc            |  auc           | 
|:---------|:--------------:|:-------------:|:--------------:|:--------------:|:--------------:|:--------------:|   
|source_rf | 0.191 ± 0.0026 | 0.073 ± 0.002 | 0.176 ± 0.0032 | 0.357 ± 0.0075 | 0.84 ± 0.0014  | 0.637 ± 0.0063 | 
|target_rf | 0.163 ± 0.0218 | 0.067 ± 0.0182| 0.138 ± 0.0339 | 0.272 ± 0.0537 | 0.813 ± 0.036  | 0.591 ± 0.0399 | 

La grande question de l'apprentissage par transfert à résoudre est la suivante : **Comment combiner les données cibles à faible biais et à haute variance avec les données sources à faible biais et à haute variance**. D'autres questions auxquelles nous nous attendons à voir des réponses :

1. Peut-on faire un meilleur prétraitement (amputation des données manquantes, utilisation du temps d'une manière plus intelligente) dans l'extracteur de caractéristiques ?
2. Normalement, les données d'arrière-plan (bonnes instances) ne participent pas au scoring, mais elles peuvent informer le classifieur du changement de distribution. Comment utiliser au mieux cette information ?

## Local testing (before submission)

You submission will contain a `feature_extractor.py` implementing a FeatureExtractor class with a `transform` function (no `fit`) and a `classifier.py` implementing a Classifier class with a `fit` and `predict_proba` functions as in the starting kit. You should place it in the `submission/<submission_name>` folder in your RAMP kit folder. To test your submission, go to your RAMP kit folder in the terminal and type
```
ramp-test --submission <submission_name>
```
It will train and test your submission much like we did it above in this notebook, and print the foldwise and summary scores. You can try it also in this notebook:

In [ ]:
!ramp-test --submission prepare_rf

If you want to have a local leaderboard, use the `--save-output` option when running `ramp-test`, then try `ramp-show leaderboard` with different options. For example:
```
ramp-show leaderboard --mean --metric "['ap','auc']" --step "['valid','test']" --precision 3
```
and
```
ramp-show leaderboard --bagged --metric "['auc']"
```

RAMP also has an experimental hyperopt feature, with random grid search implemented. If you want to use it, type
```
ramp-hyperopt --help
```
and check out the example submission [here](https://github.com/ramp-kits/titanic/tree/hyperopt/submissions/starting_kit_h).